In [1]:
import numpy as np
from qiskit.circuit.library import EfficientSU2
from qiskit.opflow import Y, X, Z, Gradient, StateFn, OperatorStateFn

from qiskit.quantum_info import Statevector, Operator
from qiskit import QuantumCircuit
from qiskit.compiler import transpile

#from qiskit.aqua.operators.gradients import Gradient

In [2]:
operator = X ^ X ^ X
ansatz = EfficientSU2(3, reps=1)
values = list(range(ansatz.num_parameters))

In [3]:
input_qc.compose(ansatz).draw()

NameError: name 'input_qc' is not defined

In [4]:
operator = Statevector.from_label('101').to_operator()
print(type(operator))

<class 'qiskit.quantum_info.operators.operator.Operator'>


In [5]:
OperatorStateFn(operator)

OperatorStateFn(Operator([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2, 2), output_dims=(2, 2, 2)), coeff=1.0, is_measurement=False)

In [6]:
input_state = np.array([0,1,0,0,0,0,0,0], dtype=np.complex_)
input_state = Statevector(input_state / np.linalg.norm(input_state))
input_qc = QuantumCircuit(3)
input_qc.initialize(input_state)

new_ansatz = input_qc.compose(ansatz)
new_ansatz = transpile(new_ansatz, optimization_level=1)
print(new_ansatz.draw())

expectation = ~OperatorStateFn(operator) @ StateFn(new_ansatz)
#gradient = Gradient(grad_method='param_shift')#.convert(expectation)
#print(np.real(gradient.bind_parameters(dict(zip(ansatz.parameters, values))).eval()))

gradient = Gradient(grad_method='param_shift').convert(expectation)
print(np.real(gradient.bind_parameters(dict(zip(ansatz.parameters, values))).eval()))

     ┌──────────────────────────────┐»
q_0: ┤0                             ├»
     │                              │»
q_1: ┤1 Initialize(0,1,0,0,0,0,0,0) ├»
     │                              │»
q_2: ┤2                             ├»
     └──────────────────────────────┘»
«     ┌──────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                             ├
«     │                                                                              │
«q_1: ┤1 EfficientSU2(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5],θ[6],θ[7],θ[8],θ[9],θ[10],θ[11]) ├
«     │                                                                              │
«q_2: ┤2                                                                             ├
«     └──────────────────────────────────────────────────────────────────────────────┘


AttributeError: 'Operator' object has no attribute 'primitive_strings'

In [7]:
inverse_circ = QuantumCircuit(3)
for inst, qargs, cargs in reversed(new_ansatz._data):
    print(inst)
    try:
        print(inst.inverse())
    except:
        pass
    inverse_circ._append(inst.inverse(), qargs, cargs)

Instruction(name='EfficientSU2', num_qubits=3, num_clbits=0, params=[ParameterVectorElement(θ[0]), ParameterVectorElement(θ[1]), ParameterVectorElement(θ[2]), ParameterVectorElement(θ[3]), ParameterVectorElement(θ[4]), ParameterVectorElement(θ[5]), ParameterVectorElement(θ[6]), ParameterVectorElement(θ[7]), ParameterVectorElement(θ[8]), ParameterVectorElement(θ[9]), ParameterVectorElement(θ[10]), ParameterVectorElement(θ[11])])
Instruction(name='EfficientSU2_dg', num_qubits=3, num_clbits=0, params=[ParameterVectorElement(θ[0]), ParameterVectorElement(θ[1]), ParameterVectorElement(θ[2]), ParameterVectorElement(θ[3]), ParameterVectorElement(θ[4]), ParameterVectorElement(θ[5]), ParameterVectorElement(θ[6]), ParameterVectorElement(θ[7]), ParameterVectorElement(θ[8]), ParameterVectorElement(θ[9]), ParameterVectorElement(θ[10]), ParameterVectorElement(θ[11])])
Instruction(name='initialize', num_qubits=3, num_clbits=0, params=[0j, (1+0j), 0j, 0j, 0j, 0j, 0j, 0j])


CircuitError: "Invalid param type <class 'complex'> for gate initialize_dg."

In [8]:
qc = QuantumCircuit(3)
qc.initialize(input_state)
#qc.inverse()
#qc = transpile(qc, optimization_level=3)
#qc.inverse()
#qc.draw()

In [9]:
qc.decompose().inverse()

CircuitError: 'inverse() not implemented for reset.'

In [10]:
from qiskit.transpiler.passes import RemoveResetInZeroState
stateprep = RemoveResetInZeroState()(qc.decompose())

In [11]:
stateprep.draw()

┌──────────────────┐
q_0: ┤0                 ├
     │                  │
q_1: ┤1 disentangler_dg ├
     │                  │
q_2: ┤2                 ├
     └──────────────────┘

In [12]:
stateprep.inverse().draw()

┌───────────────┐
q_0: ┤0              ├
     │               │
q_1: ┤1 disentangler ├
     │               │
q_2: ┤2              ├
     └───────────────┘

In [13]:
a = []
a is None

False